In [8]:
#!/usr/bin/env python3
import random
from typing import List, Sequence, Optional

# ============================================
# Bits and helpers
# ============================================

def bytes_to_bits(b: bytes) -> List[int]:
    return [(b[i // 8] >> (7 - (i % 8))) & 1 for i in range(8 * len(b))]

def bits_to_bytes(bits: Sequence[int]) -> bytes:
    if len(bits) % 8 != 0:
        raise ValueError("bits length must be multiple of 8")
    out = bytearray(len(bits) // 8)
    for i, bit in enumerate(bits):
        out[i // 8] = (out[i // 8] << 1) | (bit & 1)
    return bytes(out)

def permute(bits: Sequence[int], table: Sequence[int]) -> List[int]:
    # Table positions are 1-indexed
    return [bits[i - 1] for i in table]

def left_rotate(lst: Sequence[int], n: int) -> List[int]:
    n %= len(lst)
    return list(lst[n:] + lst[:n])

def inverse_permutation(table: Sequence[int]) -> List[int]:
    inv = [0] * len(table)
    for i, val in enumerate(table, start=1):
        inv[val - 1] = i
    return inv

# ============================================
# Dynamic table generation
# ============================================

def generate_sbox(rng: random.Random) -> List[List[int]]:
    # 6-bit input -> 4-bit output; rows: 4, cols: 16. Each row is a permutation of 0..15
    return [rng.sample(range(16), 16) for _ in range(4)]

def generate_all_sboxes(rng: random.Random) -> List[List[List[int]]]:
    return [generate_sbox(rng) for _ in range(8)]

def generate_permutation(n: int, rng: random.Random) -> List[int]:
    perm = list(range(1, n + 1))
    rng.shuffle(perm)
    return perm

def generate_IP_and_FP(rng: random.Random):
    IP = generate_permutation(64, rng)
    FP = inverse_permutation(IP)
    return IP, FP

def generate_P(rng: random.Random) -> List[int]:
    return generate_permutation(32, rng)

def generate_PC1(rng: random.Random) -> List[int]:
    # Drop parity positions (8,16,...,64) then shuffle remaining 56
    key_positions = [i for i in range(1, 65) if i % 8 != 0]
    rng.shuffle(key_positions)
    return key_positions  # length 56

def generate_PC2(rng: random.Random) -> List[int]:
    positions = list(range(1, 57))
    rng.shuffle(positions)
    return positions[:48]  # length 48

def generate_rotations(rng: random.Random, rounds: int = 16, bias_two: float = 0.6) -> List[int]:
    # Rotation schedule with values in {1,2}
    return [2 if rng.random() < bias_two else 1 for _ in range(rounds)]

def generate_E_like_expand(rng: random.Random) -> List[int]:
    # DES-style neighbor expansion from 32 -> 48 with minor random swaps
    E = []
    for k in range(8):
        start = 4 * k + 1
        block = [start, start + 1, start + 2, start + 3]
        left_neighbor = block[0] - 1 if block[0] > 1 else 32
        right_neighbor = block[-1] + 1 if block[-1] < 32 else 1
        E.extend([left_neighbor] + block + [right_neighbor])
    swaps = rng.randint(0, 2)
    for _ in range(swaps):
        i, j = rng.randrange(48), rng.randrange(48)
        E[i], E[j] = E[j], E[i]
    return E  # length 48, values 1..32, with repeats

# ============================================
# Core cipher components (DES-like)
# ============================================

def sbox_substitution(bits48: Sequence[int], SBOXES: Sequence[Sequence[Sequence[int]]]) -> List[int]:
    if len(bits48) != 48 or len(SBOXES) != 8:
        raise ValueError("Expected 48 bits and 8 S-boxes")
    out: List[int] = []
    for s in range(8):
        block = bits48[6*s:6*(s+1)]
        row = (block[0] << 1) | block[5]  # 0..3
        col = (block[1] << 3) | (block[2] << 2) | (block[3] << 1) | block[4]  # 0..15
        val = SBOXES[s][row][col]  # 0..15
        out.extend([(val >> 3) & 1, (val >> 2) & 1, (val >> 1) & 1, val & 1])
    return out  # 32 bits

def round_F(right32: Sequence[int],
            subkey48: Sequence[int],
            E: Sequence[int], P: Sequence[int],
            SBOXES: Sequence[Sequence[Sequence[int]]]) -> List[int]:
    expanded = permute(right32, E)           # 48 bits
    xored = [a ^ b for a, b in zip(expanded, subkey48)]
    sb = sbox_substitution(xored, SBOXES)    # 32 bits
    return permute(sb, P)                    # 32 bits

def generate_subkeys(key8: bytes,
                     PC1: Sequence[int], PC2: Sequence[int],
                     ROTATIONS: Sequence[int]) -> List[List[int]]:
    if len(key8) != 8:
        raise ValueError("Key must be 8 bytes (64 bits incl. parity)")
    key_bits = bytes_to_bits(key8)           # 64 bits
    key56 = permute(key_bits, PC1)           # 56 bits
    C = key56[:28]
    D = key56[28:]
    subkeys: List[List[int]] = []
    for r in range(16):
        C = left_rotate(C, ROTATIONS[r])
        D = left_rotate(D, ROTATIONS[r])
        CD = C + D                            # 56 bits
        subkeys.append(permute(CD, PC2))      # 48 bits
    return subkeys

# ============================================
# Block encrypt/decrypt
# ============================================

def des_block_encrypt(block8: bytes,
                      key8: bytes,
                      IP: Sequence[int], FP: Sequence[int],
                      E: Sequence[int], P: Sequence[int],
                      SBOXES: Sequence[Sequence[Sequence[int]]],
                      PC1: Sequence[int], PC2: Sequence[int],
                      ROTATIONS: Sequence[int]) -> bytes:
    if len(block8) != 8:
        raise ValueError("Block must be 8 bytes")
    subkeys = generate_subkeys(key8, PC1, PC2, ROTATIONS)
    bits = bytes_to_bits(block8)              # 64 bits
    ip = permute(bits, IP)
    L = ip[:32]
    R = ip[32:]
    for i in range(16):
        L, R = R, [l ^ f for l, f in zip(L, round_F(R, subkeys[i], E, P, SBOXES))]
    preoutput = R + L                         # Feistel swap
    fp = permute(preoutput, FP)
    return bits_to_bytes(fp)

def des_block_decrypt(block8: bytes,
                      key8: bytes,
                      IP: Sequence[int], FP: Sequence[int],
                      E: Sequence[int], P: Sequence[int],
                      SBOXES: Sequence[Sequence[Sequence[int]]],
                      PC1: Sequence[int], PC2: Sequence[int],
                      ROTATIONS: Sequence[int]) -> bytes:
    if len(block8) != 8:
        raise ValueError("Block must be 8 bytes")
    subkeys = generate_subkeys(key8, PC1, PC2, ROTATIONS)
    bits = bytes_to_bits(block8)
    ip = permute(bits, IP)
    L = ip[:32]
    R = ip[32:]
    for i in range(16):
        L, R = R, [l ^ f for l, f in zip(L, round_F(R, subkeys[15 - i], E, P, SBOXES))]
    preoutput = R + L
    fp = permute(preoutput, FP)
    return bits_to_bytes(fp)

# ============================================
# CBC mode with PKCS#7 padding (single mode)
# ============================================

def pkcs7_pad(data: bytes, block_size: int = 8) -> bytes:
    padlen = block_size - (len(data) % block_size)
    return data + bytes([padlen] * padlen)

def pkcs7_unpad(data: bytes) -> bytes:
    if not data:
        raise ValueError("Invalid padding: empty data")
    padlen = data[-1]
    if padlen == 0 or padlen > len(data) or data[-padlen:] != bytes([padlen] * padlen):
        raise ValueError("Invalid PKCS#7 padding")
    return data[:-padlen]

def cbc_encrypt(data: bytes, key8: bytes, iv8: bytes,
                IP, FP, E, P, SBOXES, PC1, PC2, ROTATIONS) -> bytes:
    if len(iv8) != 8:
        raise ValueError("IV must be 8 bytes")
    data = pkcs7_pad(data, 8)
    out = bytearray()
    prev = iv8
    for i in range(0, len(data), 8):
        block = bytes(a ^ b for a, b in zip(data[i:i+8], prev))
        ct = des_block_encrypt(block, key8, IP, FP, E, P, SBOXES, PC1, PC2, ROTATIONS)
        out.extend(ct)
        prev = ct
    return bytes(out)

def cbc_decrypt(data: bytes, key8: bytes, iv8: bytes,
                IP, FP, E, P, SBOXES, PC1, PC2, ROTATIONS) -> bytes:
    if len(iv8) != 8 or len(data) % 8 != 0:
        raise ValueError("IV must be 8 bytes and data multiple of 8")
    out = bytearray()
    prev = iv8
    for i in range(0, len(data), 8):
        pt_block = des_block_decrypt(data[i:i+8], key8, IP, FP, E, P, SBOXES, PC1, PC2, ROTATIONS)
        out.extend(bytes(a ^ b for a, b in zip(pt_block, prev)))
        prev = data[i:i+8]
    return pkcs7_unpad(bytes(out))

# ============================================
# Configuration generator (fix seed for reproducibility)
# ============================================

def generate_dynamic_des_config(seed: Optional[int] = None):
    rng = random.Random(seed)
    SBOXES = generate_all_sboxes(rng)
    P = generate_P(rng)
    IP, FP = generate_IP_and_FP(rng)
    PC1 = generate_PC1(rng)
    PC2 = generate_PC2(rng)
    ROTATIONS = generate_rotations(rng, rounds=16, bias_two=0.6)
    E = generate_E_like_expand(rng)
    return {
        "SBOXES": SBOXES, "P": P, "IP": IP, "FP": FP,
        "PC1": PC1, "PC2": PC2, "ROTATIONS": ROTATIONS, "E": E,
        "seed": seed
    }

# ============================================
# Main program: CBC with user-provided plaintext, key, IV
# ============================================

def parse_hex_input(prompt: str, expected_len: int) -> Optional[bytes]:
    s = input(prompt).strip()
    if not s:
        return None  # signal default
    try:
        b = bytes.fromhex(s)
    except ValueError:
        raise ValueError(f"Invalid hex input: {s}")
    if len(b) != expected_len:
        raise ValueError(f"Expected {expected_len} bytes, got {len(b)}")
    return b

def preview_config(cfg: dict):
    # Print useful preview of dynamic tables (not the full matrices)
    print("\n--- Configuration preview ---")
    print(f"Seed: {cfg['seed']}")
    print(f"IP (first 16): {cfg['IP'][:16]}")
    print(f"P  (first 16): {cfg['P'][:16]}")
    print(f"ROTATIONS: {cfg['ROTATIONS']}")
    # Show first S-box, first row
    sbox0_row0 = cfg['SBOXES'][0][0]
    print(f"SBOX[0][row 0] (first 16 vals): {sbox0_row0}")
    # Show PC1/PC2 lengths to confirm shape
    print(f"PC1 length: {len(cfg['PC1'])}, PC2 length: {len(cfg['PC2'])}, E length: {len(cfg['E'])}")
    print("--- End preview ---\n")

if __name__ == "__main__":
    # Robust seed parsing: accept decimal or 0xHEX; fallback to None
    seed_str = input("Enter seed (int or 0xHEX; blank for random): ").strip()
    seed: Optional[int] = None
    if seed_str:
        try:
            seed = int(seed_str, 0)  # base 0 allows decimal or hex (e.g., 0x2A)
        except ValueError:
            print(f"Invalid seed '{seed_str}'. Using random seed.")

    cfg = generate_dynamic_des_config(seed=seed)
    SBOXES = cfg["SBOXES"]; P = cfg["P"]
    IP = cfg["IP"]; FP = cfg["FP"]
    PC1 = cfg["PC1"]; PC2 = cfg["PC2"]
    ROTATIONS = cfg["ROTATIONS"]; E = cfg["E"]

    # Show a short preview so runs are informative
    preview_config(cfg)

    # Defaults (user can override)
    default_key_hex = "133457799BBCDFF1"      # 8 bytes
    default_iv_hex  = "0001020304050607"      # 8 bytes

    print("Provide key and IV as hex. Press Enter to use defaults.")
    key = parse_hex_input(f"Key (8 bytes hex) [default {default_key_hex}]: ", 8)
    iv  = parse_hex_input(f"IV  (8 bytes hex) [default {default_iv_hex}]: ", 8)
    if key is None:
        key = bytes.fromhex(default_key_hex)
    if iv is None:
        iv = bytes.fromhex(default_iv_hex)

    plaintext_str = input("\nEnter plaintext: ")
    plaintext = plaintext_str.encode("utf-8")

    # Encrypt (CBC)
    ciphertext = cbc_encrypt(plaintext, key, iv, IP, FP, E, P, SBOXES, PC1, PC2, ROTATIONS)

    # Decrypt (CBC)
    recovered = cbc_decrypt(ciphertext, key, iv, IP, FP, E, P, SBOXES, PC1, PC2, ROTATIONS)
    try:
        recovered_text = recovered.decode("utf-8")
        decode_ok = True
    except UnicodeDecodeError:
        recovered_text = recovered  # show raw bytes if decoding fails
        decode_ok = False

    # Display useful run information
    print("\n=== Run summary ===")
    print(f"Seed used: {cfg['seed']}")
    print(f"Key (hex): {key.hex().upper()}")
    print(f"IV  (hex): {iv.hex().upper()}")
    print(f"Plaintext length (bytes): {len(plaintext)}")
    print(f"Ciphertext length (bytes): {len(ciphertext)}")
    print(f"Ciphertext (hex): {ciphertext.hex().upper()}")
    print("Recovered:", recovered_text if decode_ok else f"<{len(recovered)} raw bytes>")
    print(f"Round-trip OK: {recovered == plaintext}")
    print("====================\n")


--- Configuration preview ---
Seed: 45
IP (first 16): [3, 19, 59, 23, 39, 8, 28, 5, 9, 16, 13, 27, 14, 63, 17, 10]
P  (first 16): [25, 26, 22, 13, 29, 17, 24, 12, 30, 14, 6, 7, 21, 3, 2, 20]
ROTATIONS: [2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1]
SBOX[0][row 0] (first 16 vals): [8, 6, 7, 4, 1, 12, 5, 0, 11, 3, 15, 9, 2, 10, 13, 14]
PC1 length: 56, PC2 length: 48, E length: 48
--- End preview ---

Provide key and IV as hex. Press Enter to use defaults.

=== Run summary ===
Seed used: 45
Key (hex): 133457799BBCDFF1
IV  (hex): 0001020304050607
Plaintext length (bytes): 11
Ciphertext length (bytes): 16
Ciphertext (hex): E8CAC4DED6295EBC98060A214BEA814F
Recovered: Hello World
Round-trip OK: True

